In [1]:
import os
import time
from collections import Counter

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from PIL import Image

import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
!pip install torchsummary
from torchsummary import summary

from sklearn.utils import resample
from sklearn.metrics import accuracy_score
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier

from tqdm.notebook import tqdm

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
train_df = pd.read_csv('/kaggle/input/heartbeat/mitbih_train.csv', header = None )
test_df = pd.read_csv('/kaggle/input/heartbeat/mitbih_test.csv', header = None )
train_df.head()

,0,1,2,3,4,5,6,7,8,9,...,178,179,180,181,182,183,184,185,186,187
0,0.977941,0.926471,0.681373,0.245098,0.154412,0.191176,0.151961,0.085784,0.058824,0.049020,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.960114,0.863248,0.461538,0.196581,0.094017,0.125356,0.099715,0.088319,0.074074,0.082621,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1.000000,0.659459,0.186486,0.070270,0.070270,0.059459,0.056757,0.043243,0.054054,0.045946,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.925414,0.665746,0.541436,0.276243,0.196133,0.077348,0.071823,0.060773,0.066298,0.058011,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.967136,1.000000,0.830986,0.586854,0.356808,0.248826,0.145540,0.089202,0.117371,0.150235,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [3]:
X_train= train_df.loc[:,:186]
y_train=np.array(train_df.loc[:,187:])
X_train.shape,y_train.shape 

((87554, 187), (87554, 1))

In [4]:
X_test= test_df.loc[:,:186]
y_test=np.array(test_df.loc[:,187:])
X_test.shape,y_test.shape 

((21892, 187), (21892, 1))

In [5]:
"""X_train = X_train.drop([186,185,184,183,182], axis = 1)
X_test = X_test.drop([186,185,184,183,182], axis = 1)
X_train"""

'X_train = X_train.drop([186,185,184,183,182], axis = 1)\nX_test = X_test.drop([186,185,184,183,182], axis = 1)\nX_train'

In [6]:

X_train = X_train.apply(lambda x: Image.fromarray(x.values.reshape(11, 17), 'L'), axis=1)
X_test = X_test.apply(lambda x: Image.fromarray(x.values.reshape(11, 17), 'L'), axis=1)

In [7]:
y_train = np.array(y_train).astype(int).squeeze()
y_test = np.array(y_test).astype(int).squeeze()
y_train.shape , y_test.shape

((87554,), (21892,))

In [8]:
class ImageDataset(Dataset):
    def __init__(self, images_series, labels, transform=None):
        self.images_series = images_series
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.images_series)

    def __getitem__(self, idx):
        image = self.images_series.iloc[idx]

        if self.transform:
            image = self.transform(image)

        label = self.labels[idx]

        return image, label

In [9]:
img_size = 112 
lr = 0.001  
num_epochs = 24  
batch_size = 256


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Updated Data Augmentations
test_transform = transforms.Compose([
    transforms.Resize((img_size, img_size)),
    transforms.ToTensor(),
])

# Data augmentation transformations
train_transform = transforms.Compose([
    transforms.Resize((img_size, img_size)),
    transforms.ToTensor(),
])


# Assuming you have a custom ImageDataset class, you can create train and test datasets like this:
train_dataset = ImageDataset(X_train, y_train, transform=train_transform)
test_dataset = ImageDataset(X_test, y_test, transform=test_transform)

# Use a smaller batch size to reduce GPU memory usage
train_loader = torch.utils.data.DataLoader(
    train_dataset,
    batch_size=batch_size,
    num_workers=8,  # Adjust the number of workers based on your system resources
    shuffle=True
)
test_loader = torch.utils.data.DataLoader(
    test_dataset,
    batch_size=batch_size,
    num_workers=8,  # Adjust the number of workers based on your system resources
    shuffle=False  # No need to shuffle test data
)


/opt/conda/lib/python3.10/site-packages/torch/utils/data/dataloader.py:561: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


In [10]:
model = torchvision.models.efficientnet_b0(weights=torchvision.models.EfficientNet_B0_Weights.DEFAULT)

Downloading: "https://download.pytorch.org/models/efficientnet_b0_rwightman-3dd342df.pth" to /root/.cache/torch/hub/checkpoints/efficientnet_b0_rwightman-3dd342df.pth
100%|██████████| 20.5M/20.5M [00:00<00:00, 73.0MB/s]


In [11]:
# Initialize new output layer
model.features[0][0] = nn.Conv2d(1, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
model.features[-1].fc = nn.AdaptiveAvgPool2d(output_size=1)
model.avgpool = nn.Identity()
model.classifier.fc = nn.Linear(1000, 512)
model.classifier.fc1 = nn.Linear(512, 128)
model.classifier.fc2 = nn.Linear(128, 5)
model = model.to(device)

In [12]:
model

EfficientNet(
  (features): Sequential(
    (0): Conv2dNormActivation(
      (0): Conv2d(1, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): SiLU(inplace=True)
    )
    (1): Sequential(
      (0): MBConv(
        (block): Sequential(
          (0): Conv2dNormActivation(
            (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
            (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (2): SiLU(inplace=True)
          )
          (1): SqueezeExcitation(
            (avgpool): AdaptiveAvgPool2d(output_size=1)
            (fc1): Conv2d(32, 8, kernel_size=(1, 1), stride=(1, 1))
            (fc2): Conv2d(8, 32, kernel_size=(1, 1), stride=(1, 1))
            (activation): SiLU(inplace=True)
            (scale_activation): Sigmoid()
          )
          (2): Conv2dNormActivat

In [13]:
import time
import torch
from tqdm import tqdm

def train_model(model, train_dataloader, test_dataloader, criterion, optimizer, device, num_epochs=50):
    since = time.time()
    best_acc = 0.0
    save_path = 'best_model.pth'  # Specify the path to save the best model

    for epoch in tqdm(range(num_epochs)):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Training phase
        model.train()
        running_train_loss = 0.0
        running_train_corrects = 0

        for inputs, labels in train_dataloader:
            inputs = inputs.to(device)
            labels = labels.to(device)

            optimizer.zero_grad()

            outputs = model(inputs)
            loss = criterion(outputs, labels)

            _, preds = torch.max(outputs, 1)

            loss.backward()
            optimizer.step()

            running_train_loss += loss.item() * inputs.size(0)
            running_train_corrects += torch.sum(preds == labels.data)

        train_loss = running_train_loss / len(train_dataloader.dataset)
        train_acc = running_train_corrects.double() / len(train_dataloader.dataset)

        print('Train Loss: {:.4f} Train Acc: {:.4f}'.format(train_loss, train_acc))

        # Testing phase
        model.eval()
        running_test_loss = 0.0
        running_test_corrects = 0

        with torch.no_grad():
            for inputs, labels in test_dataloader:
                inputs = inputs.to(device)
                labels = labels.to(device)

                outputs = model(inputs)
                
                loss = criterion(outputs, labels)

                _, preds = torch.max(outputs, 1)

                running_test_loss += loss.item() * inputs.size(0)
                running_test_corrects += torch.sum(preds == labels.data)

        test_loss = running_test_loss / len(test_dataloader.dataset)
        test_acc = running_test_corrects.double() / len(test_dataloader.dataset)

        print('Test Loss: {:.4f} Test Acc: {:.4f}'.format(test_loss, test_acc))

        if test_acc > best_acc:
            best_acc = test_acc

            # Save the best model
            torch.save(model.state_dict(), save_path)
            print('Model saved at', save_path)

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    print('Best Acc: {:.4f}'.format(best_acc))

    # Load the best model before returning
    model.load_state_dict(torch.load(save_path))
    print('Best model loaded from', save_path)

    return model


In [14]:
optimizer = torch.optim.AdamW(model.parameters(), lr=lr, weight_decay=0.0001)

# Setup the loss function
criterion = nn.CrossEntropyLoss()

# Train model
model = train_model(model, train_loader,test_loader, criterion, optimizer, device , num_epochs)

  0%|          | 0/5 [00:00<?, ?it/s]

Epoch 0/4
----------
Train Loss: 0.3190 Train Acc: 0.9136
Test Loss: 0.2120 Test Acc: 0.9445
Epoch 1/4
----------
Train Loss: 0.1841 Train Acc: 0.9494
Test Loss: 0.1944 Test Acc: 0.9470
Epoch 2/4
----------
Train Loss: 0.1568 Train Acc: 0.9560
Test Loss: 0.1768 Test Acc: 0.9506
Epoch 3/4
----------
Train Loss: 0.1379 Train Acc: 0.9605
Test Loss: 1.0138 Test Acc: 0.5102
Epoch 4/4
----------
Train Loss: 0.6359 Train Acc: 0.9100
Test Loss: 0.1981 Test Acc: 0.9502
Training complete in 7m 54s
Best Acc: 0.9506


In [19]:
def extract_features(data_loader):
    features = []
    labels = []
    with torch.no_grad():
        for inputs, batch_labels in data_loader:
            inputs = inputs.to(device).float()
            x = model.features(inputs)
            features.append(x.cpu().numpy())
            
            labels.extend([label.item() for label in batch_labels])
    features = np.vstack(features)
    return features.squeeze(), np.array(labels).astype('int')

train_features, train_labels = extract_features(train_loader)
val_features, val_labels = extract_features(test_loader)  

print(train_features.shape, train_labels.shape)
print(val_features.shape, val_labels.shape)

/opt/conda/lib/python3.10/site-packages/torch/utils/data/dataloader.py:561: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


(87554, 1280) (87554,)
(21892, 1280) (21892,)


In [20]:
clf = LGBMClassifier(objective='multiclass')

clf.fit(train_features, train_labels)

accuracy = accuracy_score(val_labels, clf.predict(val_features))
print(f'Validation Accuracy: {accuracy * 100:.2f}%')

Validation Accuracy: 95.84%


In [ ]:
from xgboost import XGBClassifier

clf = XGBClassifier(device = "cuda")

clf.fit(train_features, train_labels)

accuracy = accuracy_score(val_labels, clf.predict(val_features))
print(f'Validation Accuracy: {accuracy * 100:.2f}%')

[21:13:58] WARNING: ../src/learner.cc:767: 
Parameters: { "device" } are not used.

